In [6]:
pip install mysql-connector-python pandas

In [ ]:
import requests #librería estándar de Python para manejar peticiones HTTP de forma sencilla
import pandas as pd  #para estructurarlos
import numpy as np #trabaja con arrays numéricos de alta eficiencia y para el manejo avanzado de valores nulos
import mysql.connector #Carga la librería que permite a Python comunicarse con el servidor MySQL
from mysql.connector import errorcode #Importa módulos para identificar y manejar errores específicos de MySQL
from mysql.connector import Error 

# ====================================================================
# FASE 1: EXTRACCIÓN, TRANSFORMACIÓN Y LIMPIEZA
# ====================================================================
#definicion endpoint
url_api = "https://beta.adalab.es/resources/apis/pelis/pelis.json"
df_peliculas = pd.DataFrame() # Inicialización segura para la tabla principal
df_subtitulos_raw = pd.DataFrame() # Inicialización segura para el desanidamiento
data = []

try:
    response = requests.get(url_api) #petición GET porque solo necesitas obtener datos del servidor (la API).
    response.raise_for_status() #comprueba el código de estado HTTP (200 OK, 404 Not Found, 500 Server Error, etc.) 
    #y lanza una excepción si el código indica un fallo (4xx o 5xx).
    data = response.json()
except requests.exceptions.RequestException as e: #asegura que el programa no colapse si hay fallos de red
    print(f"Error al obtener los datos de la API: {e}")

if data: #el programa solo continúa si realmente recibió datos de la API.
    # La API devuelve una lista de diccionarios (convirtiendo) el texto plano de la respuesta de la API (que está en formato JSON) 
    # a una estructura de datos nativa de Python (lista de diccionarios).
    df_temp = pd.DataFrame(data).head(100) #conviertes rápidamente la lista de diccionarios (estructura ideal de JSON) en un objeto DataFrame de Pandas, 
    # la estructura fundamental para el análisis de datos en Python. Solo nos interesan las primeras 100 películas: filtrado o control de volumen
    
    # 1. Renombrado y Selección
    df_temp = df_temp.rename(columns={
        'id': 'id_pelicula',
        'titulo': 'titulo',
        'año': 'ano_lanzamiento',
        'duracion': 'duracion_minutos',
        'genero': 'genero',
        'adultos': 'contenido_adulto'
    })

    # 2. Desanidamiento y Selección
    # Aísla la columna de subtítulos antes de la limpieza profunda de la tabla principal.
    # Este DF es el que usaremos para el bucle de inserción de subtítulos.
    df_subtitulos_raw = df_temp[['id_pelicula', 'subtitulos']].copy() 
    
    # Seleccionamos las columnas para la tabla principal.  # Las columnas del enunciado son: Título, Año de lanzamiento, Duración (en minutos), Género, Contenido para adultos
    df_peliculas = df_temp[['id_pelicula', 'titulo', 'ano_lanzamiento', 'duracion_minutos', 'genero', 'contenido_adulto']].copy()
    
    # 3. Limpieza y Estandarización
    
    # Estandarización de nulos (np.nan, 'nan', 'NaN' -> None)
    df_peliculas = df_peliculas.replace({np.nan: None, 'nan': None, 'NaN': None})
    
    # Eliminación de filas con nulos (ya que todas las columnas son NOT NULL en MySQL)
    filas_antes = len(df_peliculas)
    df_peliculas.dropna(inplace=True)
    filas_eliminadas = filas_antes - len(df_peliculas)
    
    if filas_eliminadas > 0:
        print(f"⚠️ Atención: Se eliminaron {filas_eliminadas} filas por contener datos nulos.")
        
    print("✅ Extracción, Renombrado y Limpieza completada.")
    
else:
    print("❌ No se pudieron obtener datos para el DataFrame.") 

# ====================================================================
# FASE 2 y 3: CONEXIÓN, CREACIÓN DE TABLAS Y CARGA
# ====================================================================

nombre_bd = "Adalab_pelis"
cnx = None 

try:
    # --- Comprobación de que haya datos limpios antes de conectar ---
    if df_peliculas.empty:
        raise Exception("No hay datos válidos para la inserción. Abortando conexión a DB.")
        
    # --- CONEXIÓN INICIAL PARA CREAR DB ---
    cnx = mysql.connector.connect( # Inicializamos la variable de conexión
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',
    )
    
    mycursor = cnx.cursor() # Intento de conexión inicial sin especificar la DB para crearla #Crea un objeto cursor a través del cual se enviarán los comandos SQL al servidor MySQL
    query = f"CREATE DATABASE IF NOT EXISTS {nombre_bd}"
    mycursor.execute(query)
    print(f"💾 Base de datos '{nombre_bd}' creada/asegurada.")
    
    mycursor.close() #cerramos conexion temporalmente
    cnx.close()
    
    # --- RECONEXIÓN A LA BASE DE DATOS ESPECÍFICA ---
    cnx = mysql.connector.connect( #nos conectamos a la base de datos especifica
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',
        database=nombre_bd 
    )
    mycursor = cnx.cursor()

    # --- 4. CREACIÓN DE TABLAS ---
    TABLES = {}
    
    # 4a. Tabla PELICULAS
    TABLES['peliculas'] = (
        "CREATE TABLE `peliculas` ("
        "  `id_pelicula` INT NOT NULL,"
        "  `titulo` VARCHAR(255) NOT NULL,"
        "  `ano_lanzamiento` YEAR NOT NULL,"
        "  `duracion_minutos` INT NOT NULL,"
        "  `genero` VARCHAR(100) NOT NULL,"
        "  `contenido_adulto` VARCHAR(3) NOT NULL,"
        "  PRIMARY KEY (`id_pelicula`)" 
        ") ENGINE=InnoDB"
    )

    # 4b. Tabla SUBTITULOS (Debe crearse después de la tabla 'peliculas')
    TABLES['subtitulos'] = """
    CREATE TABLE subtitulos (
        id_subtitulo INT PRIMARY KEY AUTO_INCREMENT NOT NULL,
        id_pelicula INT NOT NULL,
        codigo_idioma VARCHAR(5) NOT NULL,
        FOREIGN KEY (id_pelicula) REFERENCES peliculas(id_pelicula)
        ON DELETE CASCADE 
        ON UPDATE CASCADE
    );
    """ 
    
    # Ejecución secuencial de creación de tablas
    for name, ddl in TABLES.items():
        try:
            print(f"⏳ Creando tabla {name}...")
            mycursor.execute(ddl)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print(f"✅ La tabla '{name}' ya existe, continuando...")
            else:
                print(f"❌ Error al crear la tabla '{name}': {err}")

    # --- 5. INSERCIÓN DE DATOS ---

    # 5a. INSERCIÓN EN PELICULAS
    insert_peliculas_query = (
        "INSERT INTO peliculas "
        "(id_pelicula, titulo, ano_lanzamiento, duracion_minutos, genero, contenido_adulto) "
        "VALUES (%s, %s, %s, %s, %s, %s)"
    )
    records_peliculas = [
        (row['id_pelicula'], row['titulo'], row['ano_lanzamiento'], row['duracion_minutos'], row['genero'], row['contenido_adulto'])
        for index, row in df_peliculas.iterrows()
    ]
    mycursor.executemany(insert_peliculas_query, records_peliculas)
    print(f"🎬 Inserción en 'peliculas' completada. {mycursor.rowcount} filas.")
    
    # 5b. DESANIDAMIENTO E INSERCIÓN EN SUBTITULOS
    
    insert_subtitulos_query = "INSERT INTO subtitulos (id_pelicula, codigo_idioma) VALUES (%s, %s)"
    records_subtitulos = []

    # Iteramos sobre el DataFrame RAW para desanidar los subtítulos
    for index, row in df_subtitulos_raw.iterrows():
        pelicula_id = row['id_pelicula']
        subtitulos_list = row["subtitulos"]
        
        # Comprobación de integridad: solo itera si es una lista válida
        if subtitulos_list is not None and isinstance(subtitulos_list, list):
            for idioma in subtitulos_list:
                # Aseguramos que el ID de la película esté en el DataFrame limpio antes de insertar
                if pelicula_id in df_peliculas['id_pelicula'].values:
                    records_subtitulos.append((pelicula_id, idioma))

    if records_subtitulos:
        mycursor.executemany(insert_subtitulos_query, records_subtitulos)
        print(f"📝 Inserción en 'subtitulos' completada. {mycursor.rowcount} filas.")

    # --- COMMIT FINAL ---
    cnx.commit()
    print("🎉 Transacción confirmada.")


except Error as e:
    print(f"❌ Error crítico de MySQL: {e}")
except Exception as e:
    print(f"❌ Error en la lógica de Python: {e}")
    
finally: #el bloque finally siempre se ejecuta, independientemente de si hay error o no
    if cnx and cnx.is_connected():
        if 'mycursor' in locals() and mycursor is not None:
             mycursor.close()
        cnx.close()
        print("🔌 Conexión a MySQL cerrada.")

✅ Extracción, Renombrado y Limpieza completada.
💾 Base de datos 'Adalab_pelis' creada/asegurada.
⏳ Creando tabla peliculas...
⏳ Creando tabla subtitulos...
🎬 Inserción en 'peliculas' completada. 100 filas.
📝 Inserción en 'subtitulos' completada. 211 filas.
🎉 Transacción confirmada.
🔌 Conexión a MySQL cerrada.


In [22]:
data

[{'id': 1,
  'titulo': 'The Godfather',
  'año': 1972,
  'duracion': 175,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 2,
  'titulo': 'The Godfather Part II',
  'año': 1974,
  'duracion': 202,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 3,
  'titulo': 'Pulp Fiction',
  'año': 1994,
  'duracion': 154,
  'genero': 'Crimen',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 4,
  'titulo': 'Forrest Gump',
  'año': 1994,
  'duracion': 142,
  'genero': 'Drama',
  'adultos': False,
  'subtitulos': ['es', 'en', 'fr']},
 {'id': 5,
  'titulo': 'The Dark Knight',
  'año': 2008,
  'duracion': 152,
  'genero': 'Acción',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 6,
  'titulo': 'Fight Club',
  'año': 1999,
  'duracion': 139,
  'genero': 'Drama',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 7,
  'titulo': 'Inception',
  'año': 2010,
  'duracion': 148,
  'genero': 'Ciencia ficción',
  'adultos

In [23]:
df_peliculas

,id_pelicula,titulo,ano_lanzamiento,duracion_minutos,genero,contenido_adulto
0,1,The Godfather,1972,175,Crimen,False
1,2,The Godfather Part II,1974,202,Crimen,False
2,3,Pulp Fiction,1994,154,Crimen,True
3,4,Forrest Gump,1994,142,Drama,False
4,5,The Dark Knight,2008,152,Acción,False
...,...,...,...,...,...,...
95,96,La vita è bella,1997,116,Drama,False
96,97,Requiem for a Dream,2000,102,Drama,True
97,98,Memento,2000,113,Thriller,True
98,99,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False


In [24]:
df_subtitulos_raw

,id_pelicula,subtitulos
0,1,"[es, en]"
1,2,"[es, en]"
2,3,"[es, en]"
3,4,"[es, en, fr]"
4,5,"[es, en]"
...,...,...
95,96,"[es, en, it]"
96,97,"[es, en]"
97,98,"[es, en]"
98,99,"[es, en]"
